# 第 11 章 时间序列

时间序列（time series）数据是一种重要的结构化数据形式，应用于多个领域，包
括金融学、经济学、生态学、神经科学、物理学等。在多个时间点观察或测量到的
任何事物都可以形成一段时间序列。很多时间序列是固定频率的，也就是说，数据
点是根据某种规律定期出现的（比如每15秒、每5分钟、每月出现一次）。时间序
列也可以是不定期的，没有固定的时间单位或单位之间的偏移量。时间序列数据的
意义取决于具体的应用场景，主要有以下几种：

- 时间戳（timestamp），特定的时刻。
- 固定时期（period），如2007年1月或2010年全年。
- 时间间隔（interval），由起始和结束时间戳表示。时期（period）可以被看做间隔（interval）的特例。
- 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入烤箱时起，每秒钟饼干的直径。

本章主要讲解前3种时间序列。许多技术都可用于处理实验型时间序列，其索引可
能是一个整数或浮点数（表示从实验开始算起已经过去的时间）。最简单也最常见
的时间序列都是用时间戳进行索引的。

## 日期和时间数据类型及工具

Python标准库包含用于日期（date）和时间（time）数据的数据类型，而且还有日
历方面的功能。我们主要会用到datetime、time以及calendar模块。  
datetime() 是用得最多的数据类型：

In [5]:
from datetime import datetime
now = datetime.now()
now
datetime(2017, 9, 25, 14, 5, 52, 72973)
now.year, now.month, now.day

(2019, 7, 11)

datetime以毫秒形式存储日期和时间。timedelta表示两个datetime对象之间的时间
差：

In [11]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(days=926, seconds=56700)

In [10]:
delta.days

926

In [9]:
delta.seconds

56700

可以给datetime对象加上（或减去）一个或多个timedelta，这样会产生一个新对
象：

In [12]:
from datetime import timedelta

start = datetime(2011, 1, 7)

start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [13]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

### 字符串和 datetime 的相互转换

利用str或strftime方法（传入一个格式化字符串），datetime对象和pandas的
Timestamp对象（稍后就会介绍）可以被格式化为字符串：

In [14]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [15]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

datetime.strptime可以用这些格式化编码将字符串转换为日期：

In [16]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [19]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

datetime.strptime是通过已知格式进行日期解析的最佳方式。但是每次都要编写格
式定义是很麻烦的事情，尤其是对于一些常见的日期格式。这种情况下，你可以用
dateutil这个第三方包中的parser.parse方法（pandas中已经自动安装好了）：

In [20]:
from dateutil.parser import parse
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil可以解析几乎所有人类能够理解的日期表示形式：

In [21]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

在国际通用的格式中，日出现在月的前面很普遍，传入dayfirst=True即可解决这个
问题：

In [22]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

pandas通常是用于处理成组日期的，不管这些日期是DataFrame的轴索引还是列。
to_datetime方法可以解析多种不同的日期表示形式。对标准日期格式（如
ISO8601）的解析非常快：

In [24]:
import pandas as pd
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

它还可以处理缺失值（None、空字符串等）：

In [25]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [26]:
idx[2]  # NaT（Not a Time）是pandas中时间戳数据的null值。

NaT

In [27]:
pd.isnull(idx)

array([False, False,  True])

## 时间序列基础

pandas最基本的时间序列类型就是以时间戳（通常以Python字符串或datatime对象
表示）为索引的Series：

In [28]:
from datetime import datetime
from pandas import Series
import numpy as np

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
        datetime(2011, 1, 7), datetime(2011, 1, 8),
        datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02   -1.368775
2011-01-05    1.218387
2011-01-07   -0.107711
2011-01-08    0.093834
2011-01-10    1.229162
2011-01-12    0.700696
dtype: float64

这些datetime对象实际上是被放在一个DatetimeIndex中的：

In [29]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

跟其他Series一样，不同索引的时间序列之间的算术运算会自动按日期对齐：

In [30]:
ts + ts[::2] # ts[::2] 是每隔两个取一个。

2011-01-02   -2.737550
2011-01-05         NaN
2011-01-07   -0.215421
2011-01-08         NaN
2011-01-10    2.458324
2011-01-12         NaN
dtype: float64

pandas用NumPy的datetime64数据类型以纳秒形式存储时间戳：

In [31]:
ts.index.dtype

dtype('<M8[ns]')

DatetimeIndex中的各个标量值是pandas的Timestamp对象：

In [32]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

只要有需要，TimeStamp可以随时自动转换为datetime对象。此外，它还可以存储
频率信息（如果有的话），且知道如何执行时区转换以及其他操作。稍后将对此进
行详细讲解。

### 索引、选取、子集构造